### Install Flask package using pip
The exclamation mark '!' is used to execute Linux commands. 
If the Flask package is installed, you can ignore the cell below.

In [ ]:
#!pip3 install flask

In [ ]:
import os

from datetime import datetime

from flask import Flask, render_template, request, jsonify

In [ ]:
app = Flask(__name__)

In [ ]:
# 定義根目錄
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)